In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import sys  

# Import rk4 py file
sys.path.insert(0, 'Codes/')
import diff_eq as DE

# Import secant method py file
sys.path.insert(0, 'C:/Users/tonba/Desktop/Masters Stuff/Projects/Numerical-Analysis-From-Scratch-Using-Python/4 Root-Finding Methods/Codes')
#from root_finding import *

import root_finding as RF

# Boundary Value Problems (BVPs)

Suppose we are trying to solve 
\
\
    $$w'' = f(x, w, w'),\qquad w(a)=c,\qquad w(b) = d$$
\
\
The function's values at the endpoint, $w(b)$, is called a *Dirichlet boundary condition*. If $w'(b)$ was given, then it is called a *Neumann boundary condition*. We won't be able to blindly use RK4 since it requires an initial condition for the velocity. A familiar problem would the free fall problem (neglecting air resistance):
\
\
    $$\frac{d^2x}{dt^2} = - g$$
\
\
We could specify $x=0$ at $t=0$ and also set the position $x = 0$ at $t = t_1$. This is a boundary value problem. We want to find the solution $x(t)$ that satisfies the Dirichlet boundary conditions.

# Shooting Method

The simplest way to solve the BVP is to turn it into a initial value problem. We make use of guessing the appropriate value for $w'(a)$ such that it satisfies the boundary conditions. We need to emphasize that guessing $w'(a)$ might not always give the correct answer. We could keep guessing until we obtain a $w'(a)$ that is just right. This sounds like we need a **root-finding problem** such as binary search, bisection method, etc.


### Example from https://www.youtube.com/watch?v=qIfxydBEdzg:
Suppose we have an ODE
\
\
    $$\frac{d^2 y}{dx^2} = f(x, y, dy/dx)$$
\
\
supplied with the boundary conditions $y(x_0) = y_0$ and $y(x_f) = y_f$. As usual we express this 2nd-order ODE as a system of ODEs
\
\
    \begin{equation}
        \frac{dy}{dx} = z, \qquad \frac{dz}{dx} = f(x, y, z)
    \end{equation}
\
\
We know that $y(x_0) = y_0$ is known but we are not supplied with the initial condition for $z$. We name the unknown initial condition as $z(x_0) = \xi$. The constraint is that $y(x_f) = y_f$. We define the function
\
\
    $$F(\xi) = y(x_f) - y_f.$$
\
\
We need to find $\xi$ such that $F(\xi) = 0$. We go back to the free fall problem. We convert it into a system of first-order ODEs
\
\
    \begin{equation}
        \frac{dx}{dt} = v, \qquad \frac{dv}{dt} = -g
    \end{equation}
\
\
with $x(t_0 = 0) = 0$ and $x(t_f=1) = 2$.

In [ ]:
g = 9.8

# t boundaries
t_0 = 0
t_f = 3

# Boundary values
x_0 = 0
x_f = 10

def f(r, t):
    x = r[0]
    v = r[1]
    fx = v
    fv = -g
    
    return np.array([fx,fv])


def shoot(xi):
    # Guess of initial values
    r0 = np.array([0, xi]) # x(0) = 0, v(0) = \xi
    
    # time-values
    t = np.linspace(t_0, t_f, 1000)
    
    # RK4, solving for x(t) and v(t) given the guesses
    xs, vs = DE.rk4(f, r0, t)
    
    # Define F
    F = vs[-1] - x_f # vs[-1] = x(t_f; \xi)
    
    return F

# We find the initial value of the velocity
v_0 = RF.riddlers_method(shoot, 1, 10)
v_0

# We check

In [ ]:
x = lambda t: -0.5*g*t**2 + v_0*t + x_0 # x(t)

x(t_f) # wrong!!!!!!

In [ ]:
g = 9.8

# t-values
t = np.linspace(t_0, t_f, 1000)

# Guess of initial values
r0 = np.array([x_0, v_0]) # x(0) = 0, v(0) = \xi

def f(r, t):
    x = r[0]
    v = r[1]
    fx = v
    fv = -g
    
    return np.array([fx,fv])

# RK4, solving for x(t) and v(t) given the guesses
xs, vs = DE.rk4(f, r0, t)


plt.figure()

plt.plot(t, x(t), '--', lw= 5, label="True")
plt.plot(t, xs, '.', label="RK4", markersize=2)
plt.xlabel(r'$t$')
plt.ylabel(r'$x(t)$')

plt.legend()

### Example

We want to solve
\
\
    \begin{gather}
        w''(x) = - \frac{30}{1 - x^2} w(x) + \frac{2x}{1 - x^2} w'(x)\\
        w(0.05) = 0.0926587109375, \qquad w(0.49) = 0.1117705085875
    \end{gather}
\
\
We convert this into a system of first-order ODEs:
    \begin{gather}
        w'(x) = v, \qquad v'(x) = - \frac{30}{1 - x^2} w(x) + \frac{2x}{1 - x^2} v\\
        w(0.05) = 0.0926587109375, \qquad v(0.49) = ?
    \end{gather}
\
\

In [ ]:
def f(r, x):
    w = r[0]
    v = r[1]
    fw = v
    fv = - 30 * w / (1 - x**2)  + (2*x*v)/(1-x**2)
    
    return np.array([fw,fv])

def shoot(xi):
    # x boundaries
    x_0 = 0.05
    x_f = 0.49
    
    # Boundary values
    w_0 = 0.0926587109375
    w_f = 0.1117705085875
    
    # Guess of initial values
    r0 = np.array([0, xi]) # x(0) = 0, v(0) = \xi
    
    # x-values
    x = np.linspace(x_0, x_f, 1000)
    
    # RK4, solving for x(t) and v(t) given the guesses
    ws, vs = rk4(f, r0, x)
    
    # Define F
    F = ws[-1] - w_f
    
    return F


# We find the initial value of the velocity
secant_method(shoot, 0, 1)

### Example: Cannon problem

# Relaxation Method

# Matrix Method

In [ ]:
?rk4